# Load the weights files from the folder ../weights

In [6]:
from sklearn.preprocessing import normalize
import numpy as np

# Substitute the weights file you have just generated
weights_0_10 = np.load("./weights/weights-0-10-NEW-mean.npy")
weights_10_20 = np.load("./weights/weights-10-20-NEW-mean.npy")
weights_20_30 = np.load("./weights/weights-20-30-NEW-mean.npy")

# Substitute the xgboost model you have generated
indexes_xgboost = np.load("./weights/indexes_xgboost.npy")

# Concatenate 3 weights files into one
weights_dfs = np.concatenate((weights_0_10, weights_10_20, weights_20_30))
# Get the absolute and normalized version of the weights
weights_dfs = normalize(abs(weights_dfs))

# Save the weights you have reproduced
np.save("./weights/weights-NEW-mean", weights_dfs)

In [77]:
dataB_7169 = sio.loadmat('/Volumes/TONY/Regeneron/Data/OriginalData/B_3labels_mean_scaled_7169.mat')

In [78]:
index_COPDAcos = [i for i in xrange(dataB_7169['X'].shape[0]) if i not in np.where(dataB_7169['Y']==1)[1]]

In [79]:
dataB_7169['X'] = dataB_7169['X'][index_COPDAcos,:]
dataB_7169['Y'] = dataB_7169['Y'][0][index_COPDAcos]
dataB_7169['Y'] = np.array([0 if i == 0 else 1 for i in dataB_7169['Y']])

sio.savemat('/Volumes/TONY/Regeneron/Data/OriginalData/B_COPDAcos_mean_scaled_7169.mat', dataB_7169)

In [82]:
dataB_7169['Y'].shape

(6491,)

In [72]:
dataB_7169['X'].shape

(10684, 7169)

In [73]:
dataB_7169['Y'].shape

(10684,)

In [60]:
index_keep = [i for i in xrange(dataB['X'].shape[1]) if i not in index_eliminatedAll]

In [61]:
dataB['columnNames'].shape

(7205,)

# Load the data

In [47]:
from scipy import io as sio

# Load the data
ourdata = sio.loadmat("./data/B_mean_2labels.mat")

inputX = ourdata['X']

# Normalize the data will lead to a worse result, as we previously presented, but a larger difference
# in performance of XGBoost and DFS
# inputX = normalize(inputX, axis=0)

inputY = ourdata['Y'][0,:]
columnNames = ourdata['columnNames']

In [48]:
inputX.shape

(10684, 7205)

# Functions that regenerate the input data X based on different standards

In [8]:
# Averaging the weights based on the sum of all the normalized weights
averagedWeight = weights_dfs.sum(axis=0)
# ranking indexes for DFS (Average)
indexes_average_dfs = np.argsort(averagedWeight)[::-1]

# Select features in rank [a, b] from average DFS model, and also the indexes of these features
def topAveDFSfeatures(a,b):
    return inputX[:, indexes_average_dfs.tolist()[a:b]], indexes_average_dfs.tolist()[a:b]

# Select the union of top N features from each of the DFS model, and also the indexes of these features
def unionDFSfeatures(n):
    indexes_union = []
    for i in xrange(30):
        indexes_union.append(np.argsort(weights_dfs[i])[::-1][:n].tolist())
        
    union = reduce(np.union1d, indexes_union).tolist()
#     print("Number of union features:", len(union))
    
    return inputX[:, union], union

# Select the intersection of top N features from each of the DFS model, and also the indexes of these features
def intersectDFSfeatures(n):
    indexes_intersect = []
    for i in xrange(30):
        indexes_intersect.append(np.argsort(weights_dfs[i])[::-1][:n].tolist())
    
    intersected = reduce(np.intersect1d, indexes_intersect).tolist()
#     print("Number of intersected features:",len(intersected))
    return inputX[:, intersected], intersected

# Select features in rank [a, b] from each of the XGBoost model, and also the indexes of these features
def topXGBoostfeatures(a,b):
    return inputX[:, indexes_xgboost.tolist()[a:b]], indexes_xgboost.tolist()[a:b]

# SSelect features in rank [a, b] from one of the DFS model (n stands for which single model), 
# and also the indexes of these features
def pickOneDFSfeatures(a,b,n):
    indexx = np.argsort(weights_dfs[n])[::-1]
    return inputX[:, indexx.tolist()[a:b]], indexx.tolist()[a:b]

# Run each models

In [8]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_score

svm = LinearSVC()

temp, _ = intersectDFSfeatures(200)
scores = cross_val_score(svm, temp, inputY, cv=5)
print("Intersect:", np.mean(scores))

temp, _ = unionDFSfeatures(20)
scores = cross_val_score(svm, temp, inputY, cv=5)
print("Union:", np.mean(scores))

temp, _ = topAveDFSfeatures(0,50)
scores = cross_val_score(svm, temp, inputY, cv=5)
print("Ave:", np.mean(scores))

temp, _ = topXGBoostfeatures(0,50)
scores = cross_val_score(svm, temp, inputY, cv=5)
print("XGBoost:", np.mean(scores))

print("Pick one TOP DFS features from the existing single models.")
for i in xrange(0, weights_dfs.shape[0]):
    temp, _ = pickOneDFSfeatures(0,50,i)
    scores = cross_val_score(svm, temp, inputY, cv=5)
    print(np.mean(scores))

('Intersect:', 0.99513296970159071)
('Union:', 0.99644317410261452)
('Ave:', 0.99663061519881191)
('XGBoost:', 0.99560104682432693)
Pick one TOP DFS features from the existing single models.
0.99560100305
0.99653706987
0.995881814337
0.995788312824
0.995881639159
0.995975622475
0.996349672548
0.995320191886
0.996256170994
0.99597549107
0.995881814337
0.995320279434
0.996069124029
0.995975447296
0.995788137604
0.995881814337
0.996162669399
0.996162581809
0.996068948891
0.995601046865
0.996443393096
0.996256170994
0.996817837342
0.996537069829
0.996256170953
0.995975491152
0.99606894885
0.996536938425
0.996349891501
0.995881858193


# The heat map for weights

In [12]:
import plotly.plotly as py
import plotly.graph_objs as go
import plotly

plotly.tools.set_credentials_file(username='tonyabracadabra', api_key='6gs9i5iec7')

data = [
    go.Heatmap(
        z = weights_dfs
    )
]

py.iplot(data, filename='Weights from DFS stacked')

In [70]:
# Run SVM with top features from DFS union model
scoresUnion = []
n_unions = []
for i in xrange(1,50):
    keke, union = unionDFSfeatures(i)
    if len(n_unions) == 0 or len(union) != n_unions[-1]:
        n_unions.append(len(union))
        scoresUnion.append(np.mean(cross_val_score(svm, keke, inputY, cv=5)))
        print("Union:", scoresUnion[-1])
    
traceUnion = go.Scatter(
    x = n_unions,
    y = scoresUnion,
    mode = 'lines+markers',
    name = 'lines+markers'
)

print("DFS Union has been ran.")

"""
    Below models are ran with maximu n == n_unions[-1]
"""

# Run SVM with top features from DFS average model
scoresAve = []
for n in xrange(1, n_unions[-1]):
    temp, _ = topAveDFSfeatures(0,n)
    scoresAve.append(np.mean(cross_val_score(svm, temp, inputY, cv=5)))
    print("Ave:", scoresAve[-1])

traceAve = go.Scatter(
    x = range(1, n_unions[-1]),
    y = scoresAve,
    mode = 'lines+markers',
    name = 'lines+markers'
)

print("DFS average has been ran.")

# Run SVM with top features from DFS single models
tracesSingle = []
for i in xrange(5):
    print("Has ran a single DFS model:"+str(i))
    scoresNormal = []
    for n in xrange(1, n_unions[-1]):
        temp, _ = pickOneDFSfeatures(0,n,i)
        scoresNormal.append(np.mean(cross_val_score(svm, temp, inputY, cv=5)))
        
    trace = go.Scatter(
        x = range(1, n_unions[-1]),
        y = scoresNormal,
        mode = 'lines+markers',
        name = 'lines+markers'
    )
    tracesSingle.append(trace)
    
scoresXgboost = []
for n in xrange(1, n_unions[-1]):
    temp, _ = topXGBoostfeatures(0,n)
    scoresXgboost.append(np.mean(cross_val_score(svm, temp, inputY, cv=5)))
    print("XGBoost:", scoresXgboost[-1])

traceXGB = go.Scatter(
    x = range(1, n_unions[-1]),
    y = scoresXgboost,
    mode = 'lines+markers',
    name = 'lines+markers'
)

print("XGBoost model has been ran.")

In [69]:
a = [traceUnion,traceAve,traceXGB]
a.extend(tracesSingle)

py.iplot(a, filename='plot')

# Compare features [0,10], [10,20], [20,30] ... [90,100]

In [77]:
scoresXgboost_range = []
for i in xrange(20):
    temp, _ = topXGBoostfeatures(i,i+10)
    scoresXgboost_range.append(np.mean(cross_val_score(svm, temp, inputY, cv=5)))
    print("XGBoost:", scoresXgboost_range[-1])

traceXGB_range = go.Scatter(
    x = range(20),
    y = scoresXgboost_range,
    mode = 'lines+markers',
    name = 'lines+markers'
)

scoresAve_range = []
for i in xrange(20):
    temp, _ = topAveDFSfeatures(i,i+10)
    scoresAve_range.append(np.mean(cross_val_score(svm, temp, inputY, cv=5)))
    print("Ave:", scoresAve_range[-1])

traceAve_range = go.Scatter(
    x = range(20),
    y = scoresAve_range,
    mode = 'lines+markers',
    name = 'lines+markers'
)

# Run SVM with top features from DFS single models
tracesSingle_range = []
for i in xrange(5):
    print("Has ran a single DFS model:"+str(i))
    scoresNormal = []
    for j in xrange(20):
        temp, _ = pickOneDFSfeatures(j,j+10,i)
        scoresNormal.append(np.mean(cross_val_score(svm, temp, inputY, cv=5)))
        
    trace = go.Scatter(
        x = range(20),
        y = scoresNormal,
        mode = 'lines+markers',
        name = 'lines+markers'
    )
    tracesSingle_range.append(trace)

('XGBoost:', 0.99438390607216576)
('XGBoost:', 0.98240388071427309)
('XGBoost:', 0.97781779327135854)
('XGBoost:', 0.97725621451228617)
('XGBoost:', 0.97800501537381934)
('XGBoost:', 0.97603946789146945)
('XGBoost:', 0.98025141804039273)
('XGBoost:', 0.9808131720190465)
('XGBoost:', 0.83620461202334284)
('XGBoost:', 0.7792956608839221)
('XGBoost:', 0.80531506660464769)
('XGBoost:', 0.75224440983586516)
('XGBoost:', 0.77592404282206895)
('XGBoost:', 0.74775191129068774)
('XGBoost:', 0.74148007998623489)
('XGBoost:', 0.74129294547307123)
('XGBoost:', 0.74381950215863235)
('XGBoost:', 0.73474214217478018)
('XGBoost:', 0.68251514527155899)
('XGBoost:', 0.68700825661392106)
('Ave:', 0.99466484872272543)
('Ave:', 0.98390121979240841)
('Ave:', 0.98240374971970401)
('Ave:', 0.98015690910664444)
('Ave:', 0.97285651635623527)
('Ave:', 0.9883001729270946)
('Ave:', 0.98362067143714127)
('Ave:', 0.98324627096538764)
('Ave:', 0.98193571645309707)
('Ave:', 0.9863351072473604)
('Ave:', 0.9720145647893

In [78]:
b = [traceAve_range,traceXGB_range]
b.extend(tracesSingle_range)

py.iplot(b, filename='plot')

# The function that retrieves the names for the features

In [2]:
# ICD-9 codes mapping
import pandas as pd

dictionary = pd.read_csv("./data/BinaryTraitMatrix_V2_F50K_DD_20160523.csv")
codemapping = {i:j for i, j in zip(dictionary["FIELD_NAME"].values, dictionary["FIELD_DESCRIPTION"].values)}

def getFeatureNames(indexes):
    return [codemapping[str(i[0])] for i in columnNames[0][indexes]]

In [3]:
a = [i[0] for i in codemapping.items() if 'Emphysema' in i[1]]
b = [i[0] for i in codemapping.items() if 'emphysema' in i[1]]
c = [i[0] for i in codemapping.items() if 'asthma' in i[1]]
d = [i[0] for i in codemapping.items() if 'Asthma' in i[1]]
e = [i[0] for i in codemapping.items() if 'Chronic bronchitis' in i[1]]
f = [i[0] for i in codemapping.items() if 'chronic bronchitis' in i[1]]

In [4]:
icd9_eliminated = a+b+c+d+e+f

In [20]:
icd9_eliminated

['Phe10_J439',
 'Phe10_J43',
 'Phe9_4920',
 'Phe9_492',
 'Phe9_5181',
 'Phe10_J982',
 'Phe10_J438',
 'Phe9_4928',
 'Phe10_J454',
 'Phe10_J455',
 'Phe10_J452',
 'Phe10_J453',
 'Phe10_J459',
 'Phe9_4938',
 'Phe10_J45',
 'Phe9_4939',
 'Phe9_4930',
 'Phe9_4931',
 'Phe9_4932',
 'Phe9_493',
 'Phe9_491',
 'Phe10_J411',
 'Phe9_4911',
 'Phe9_4918',
 'Phe9_4919',
 'Phe10_J41',
 'Phe10_J42']

In [19]:
index_eliminated2

[1385,
 1373,
 1378,
 1377,
 4885,
 1365,
 4886,
 1376,
 4883,
 4888,
 4884,
 1367,
 1379,
 4882,
 1370,
 1372,
 1375,
 4880,
 1374,
 1380,
 4877,
 4879,
 4881,
 1369,
 4887,
 4874,
 4889,
 4891,
 4876,
 1368,
 1371]

In [39]:
index_eliminated = [i for i, j in enumerate(columnNames[0]) if str(j[0]) in icd9_eliminated]

In [40]:
len(index_eliminated)+len(index_eliminated2)

58

In [41]:
index_eliminatedAll = index_eliminated + index_eliminated2

In [42]:
index_eliminatedAll = np.unique(np.array(index_eliminatedAll))

58

In [46]:
index_eliminatedAll.shape

(36,)

In [57]:
featuresSelectedMySelf = np.ndarray((27,2), dtype=object)

featuresSelectedMySelf[:,0] = np.array(getFeatureNames(index_eliminated))
featuresSelectedMySelf[:,1] = index_eliminated

featuresSelectedMySelf = pd.DataFrame(featuresSelectedMySelf)

featuresSelectedMySelf.to_csv("/Volumes/TONY/Regeneron/Data/featuresSelectedMySelf.csv", header=['Feature', 'Index'])

In [58]:
np.save('/Volumes/TONY/Regeneron/Data/index_eliminated.npy', index_eliminated)

In [18]:
# np.array(getFeatureNames(indexes_average_dfs[:100]))

In [15]:
index_eliminated2 = indexes_average_dfs[:24].tolist()

In [16]:
index_eliminated2.append(4887)
index_eliminated2.append(4874)
index_eliminated2.append(4889)
index_eliminated2.append(4891)
index_eliminated2.append(4876)
index_eliminated2.append(1368)
index_eliminated2.append(1371)

In [55]:
top100 = np.ndarray((100,2), dtype=object)

In [32]:
top100[:,0] = np.array(getFeatureNames(indexes_average_dfs[:100]))
top100[:,1] = indexes_average_dfs[:100]

In [44]:
import pandas as pd

top100 = pd.DataFrame(top100)

top100.to_csv("/Volumes/TONY/Regeneron/Data/top100Features.csv", header=['Feature', 'Index'])

In [45]:
top100

,0,1
0,"ICD9 3D: Chronic airway obstruction, not elsew...",1385
1,ICD9 3D: Asthma,1373
2,"ICD9 4D: Asthma, Unspecified",1378
3,ICD9 4D: Exercise induced bronchospasm;Cough v...,1377
4,ICD10 4D: Mild persistent asthma,4885
5,ICD9 3D: Chronic bronchitis,1365
6,ICD10 4D: Moderate persistent asthma,4886
7,ICD9 4D: Chronic Obstructive Asthma,1376
8,ICD10 3D: Asthma,4883
9,ICD10 4D: Other and unspecified asthma,4888


# Find intersecting features from each model

In [ ]:
_, ave = topAveDFSfeatures(0,102)
_, xg = topXGBoostfeatures(0,102)
_, union = unionDFSfeatures(27)

# Intersection indexes
intersect_3models = np.intersect1d(np.intersect1d(ave,union),xg).tolist()
print temp.shape

temp = inputX[:,intersect_3models]

# Accuracy rate based on the three models
np.mean(cross_val_score(svm, temp, inputY, cv=5))